# Section A - Working with the RDD API

# A.1

In [1]:
from pyspark.sql import SparkSession 
spark_session = SparkSession.builder \
   .master("spark://192.168.2.156:7077") \
   .appName("alexanderSundquist_A1") \
   .config("spark.dynamicAllocation.enabled", True) \
   .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
   .config("spark.shuffle.service.enabled", False) \
   .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
   .config("spark.cores.max", 8) \
   .config("spark.driver.port",9999)\
   .config("spark.blockManager.port",10005)\
   .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 10:40:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/07 10:40:41 WARN Utils: Service 'sparkDriver' could not bind on port 9999. Attempting port 10000.
25/03/07 10:40:41 WARN Utils: Service 'sparkDriver' could not bind on port 10000. Attempting port 10001.
25/03/07 10:40:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/07 10:40:41 WARN StandaloneSchedulerBackend: Dynamic allocation enabled without spark.executor.cores explicitly set, you may get more executors allocated than expected. It's recommended to set spark.executor.cores explicitly. Please check SPARK-30299 for more details.
25/03/07 10:40:42 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 10005. Attempting port 1

In [2]:
# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

## A.1.1 & A.1.2

In [3]:
# Read Swedish and its English counterpart from HDFS
en_file = spark_context.textFile("hdfs://192.168.2.156:9000/data/europarl/europarl-v7.sv-en.en")
sv_file = spark_context.textFile("hdfs://192.168.2.156:9000/data/europarl/europarl-v7.sv-en.sv")

In [4]:
en_file.first()

'Resumption of the session'

In [5]:
sv_file.first()

'Återupptagande av sessionen'

## A.1.3

In [6]:
print("Line count Swedish file", sv_file.count())
print("Line count English file", en_file.count())

Line count Swedish file 1862234


[Stage 3:=============================>                             (1 + 1) / 2]

Line count English file 1862234


## A.1.4

In [7]:
# Count number of partitions 
en_file.getNumPartitions()

2

In [8]:
sv_file.getNumPartitions()

3

# A.2

## A.2.1

In [9]:
first_line = sv_file.first()
first_line_lower = first_line.lower()
print(first_line_lower)

[Stage 4:>                                                          (0 + 1) / 1]

återupptagande av sessionen


In [10]:
import re
def remove_special_char(file):
    return file.map(lambda line: re.sub("[^A-Za-z0-9 -åäöÅÄÖ]+",'', line))

In [11]:
def lower_case(file):
    return file.map(lambda line: line.lower())

In [12]:
def tokenize(file):
    return file.map(lambda line: line.split(" "))

In [13]:
sv_preprocessed = remove_special_char(sv_file)
sv_preprocessed = lower_case(sv_preprocessed)
sv_preprocessed = tokenize(sv_preprocessed)

en_preprocessed = remove_special_char(en_file)
en_preprocessed = lower_case(en_preprocessed)
en_preprocessed = tokenize(en_preprocessed)

## A.2.2

In [14]:
print("Swedish preprocessed: ", sv_preprocessed.take(10))
print("English preprocessed: ", en_preprocessed.take(10))

Swedish preprocessed:  [['återupptagande', 'av', 'sessionen'], ['jag', 'förklarar', 'europaparlamentets', 'session', 'återupptagen', 'efter', 'avbrottet', 'den', '17', 'december.', 'jag', 'vill', 'på', 'nytt', 'önska', 'er', 'ett', 'gott', 'nytt', 'år', 'och', 'jag', 'hoppas', 'att', 'ni', 'haft', 'en', 'trevlig', 'semester.'], ['som', 'ni', 'kunnat', 'konstatera', 'ägde', '"den', 'stora', 'år', '2000-buggen"', 'aldrig', 'rum.', 'däremot', 'har', 'invånarna', 'i', 'ett', 'antal', 'av', 'våra', 'medlemsländer', 'drabbats', 'av', 'naturkatastrofer', 'som', 'verkligen', 'varit', 'förskräckliga.'], ['ni', 'har', 'begärt', 'en', 'debatt', 'i', 'ämnet', 'under', 'sammanträdesperiodens', 'kommande', 'dagar.'], ['till', 'dess', 'vill', 'jag', 'att', 'vi,', 'som', 'ett', 'antal', 'kolleger', 'begärt,', 'håller', 'en', 'tyst', 'minut', 'för', 'offren', 'för', 'bl.a.', 'stormarna', 'i', 'de', 'länder', 'i', 'europeiska', 'unionen', 'som', 'drabbats.'], ['jag', 'ber', 'er', 'resa', 'er', 'för', 'e

[Stage 6:>                                                          (0 + 1) / 1]

English preprocessed:  [['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 

## A.2.3

In [15]:
print("Line count Swedish file", sv_preprocessed.count())
print("Line count English file", en_preprocessed.count())

Line count Swedish file 1862234


[Stage 8:=============================>                             (1 + 1) / 2]

Line count English file 1862234


# A.3

## A.3.1 and A.3.2

In [16]:
import re
def remove_special_char(file):
    return file.map(lambda line: re.sub("[^A-Za-z0-9 åäöÅÄÖ]+",'', line))

def lower_case(file):
    return file.map(lambda line: line.lower())

# Easier to use flatmap to break out of lists that are kept with map()
def tokenize(file):
    return file.flatMap(lambda line: line.split(" "))

sv_pre = remove_special_char(sv_file)
sv_pre = lower_case(sv_pre)
sv_pre = tokenize(sv_pre)

en_pre = remove_special_char(en_file)
en_pre = lower_case(en_pre)
en_pre = tokenize(en_pre)

In [17]:
# MapReduce to count occurences of each word

en_words = en_pre.map(lambda w: (w,1))
en_counts = en_words.reduceByKey(lambda a, b: a + b)

sv_words = sv_pre.map(lambda w: (w,1))
sv_counts = sv_words.reduceByKey(lambda a, b: a + b)

print(en_counts.take(5))
print(sv_counts.take(5))

[('resumption', 527), ('of', 1662006), ('i', 504521), ('declare', 1386), ('european', 270336)]


[Stage 11:===================>                                      (1 + 2) / 3]

[('session', 771), ('efter', 42231), ('avbrottet', 293), ('december', 6019), ('vill', 131230)]


In [18]:
sv_words.take(5)

[('återupptagande', 1),
 ('av', 1),
 ('sessionen', 1),
 ('jag', 1),
 ('förklarar', 1)]

In [19]:
# Calculate 10 most reacurring words in sv file

sorted_sv_counts = sv_counts.sortBy(lambda w: w[1], ascending=False)
sorted_sv_counts.take(10)

[('att', 1709939),
 ('och', 1350379),
 ('i', 1054253),
 ('det', 952991),
 ('som', 917591),
 ('för', 915081),
 ('av', 740724),
 ('är', 701842),
 ('en', 636829),
 ('vi', 546072)]

In [20]:
# Calculate total words in sv file

from operator import add

sv_words = sv_file.map(lambda line: line.split(" "))

sv_word_counts = sv_words.map(lambda w: len(w))

sv_total_words = sv_word_counts.reduce(add)

print(f"total words= {sv_total_words}")  

[Stage 21:===================>                                      (1 + 2) / 3]

total words= 41604741


Out of 41 million words the most common word makes up 1.7 million. That seems reasonable to me.

In [21]:
# Calculate 10 most reacurring words in en file
sorted_en_counts = en_counts.sortBy(lambda w: w[1], ascending=False)
sorted_en_counts.take(10)

[('the', 3505175),
 ('of', 1662006),
 ('to', 1543746),
 ('and', 1318369),
 ('in', 1088902),
 ('that', 839083),
 ('is', 774941),
 ('a', 774545),
 ('for', 538192),
 ('we', 526488)]

In [22]:
# Calculate total words for en file
from operator import add

en_words = en_file.map(lambda line: line.split(" "))

en_word_counts = en_words.map(lambda w: len(w))

en_total_words = en_word_counts.reduce(add)

print(f"total words= {en_total_words}")  

[Stage 29:=============================>                            (1 + 1) / 2]

total words= 45778381


Out of 46 million words the most common word makes up 3.5 million. That seems reasonable to me.

# A.4

## A.4.1

In [23]:
sv_1 = sv_preprocessed.zipWithIndex()
en_1 = en_preprocessed.zipWithIndex()

## A.4.2

In [24]:
sv_2 = sv_1.map(lambda x: (x[1], x[0]))
en_2 = en_1.map(lambda x: (x[1], x[0]))

## A.4.3

In [25]:
sv_en_3 = sv_2.join(en_2)

In [26]:
sv_en_3.take(3)

[(205,
  (['även',
    'om',
    'jag',
    'är',
    'medveten',
    'om',
    'att',
    'detta',
    'endast',
    'är',
    'ett',
    'litet',
    'steg',
    'mot',
    'ökad',
    'transportsäkerhet',
    'ber',
    'jag',
    'er',
    'anta',
    'betänkandet.'],
   ['i',
    'do',
    'realise',
    'that',
    'this',
    'is',
    'only',
    'a',
    'small',
    'step',
    'towards',
    'increased',
    'transport',
    'safety,',
    'but',
    'i',
    'would',
    'ask',
    'you',
    'to',
    'endorse',
    'this',
    'report.'])),
 (1565,
  (['som',
    'ni',
    'vet',
    'genomförs',
    'endast',
    'omröstningar',
    'med',
    'namnupprop',
    'och',
    'elektroniska',
    'omröstningar',
    'om',
    'kollegerna',
    'begär',
    'detta.'],
   ['you',
    'know',
    'that',
    'voting',
    'by',
    'roll',
    'call',
    'or',
    'electronic',
    'checking',
    'only',
    'takes',
    'place',
    'if',
    'members',
    'of',
    'the',
 

## A.4.4

In [27]:
sv_en_4 = sv_en_3.filter(lambda x: x[1][1] != [''] and x[1][0] != [''])

In [29]:
print("Line count sv en removed empty lines", sv_en_4.count())


[Stage 37:==============================================>           (4 + 1) / 5]

Line count sv en removed empty lines 1848423


In [30]:
print("Line count sv en not removed empty lines: ", sv_en_3.count())

[Stage 39:==============================================>           (4 + 1) / 5]

Line count sv en not removed empty lines:  1862234


In [48]:
print("Number of lines removed: ", sv_en_3.count()-sv_en_4.count())

[Stage 74:==================================>                       (3 + 2) / 5]

Number of lines removed:  13811


## A.4.5

In [38]:
sv_en_5 = sv_en_4.filter(lambda x: len(x[1][1]) < 10 and len(x[1][0]) < 10) 

In [49]:
print(f"Line count sv_en_5 {sv_en_5.count()}")

[Stage 76:==============================================>           (4 + 1) / 5]

Line count sv_en_5 200796


## A.4.6

In [40]:
sv_en_6 = sv_en_5.filter(lambda x: len(x[1][1]) == len(x[1][0])) 

In [41]:
print(f"Line count sv_en_6 {sv_en_6.count()}")

[Stage 53:==============================================>           (4 + 1) / 5]

Line count sv_en_6 73844


## A.4.7

In [44]:
sv_en_7 = sv_en_6.flatMap(lambda x: zip(x[1][0], x[1][1]))

In [45]:
sv_en_7.take(3)

[('det', 'the'), ('är', 'commission'), ('vad', 'is')]

## A.4.8

In [46]:
# MapReduce to count occurences of each word pair
from operator import add

wp_map = sv_en_7.map(lambda wp: (wp,1))

wp_counts = wp_map.reduceByKey(add)

print(wp_counts.take(5))

[Stage 59:==============================================>           (4 + 1) / 5]

[(('bör', 'should'), 362), (('göra', 'do'), 267), (('det.', 'that.'), 303), (('värd', 'the'), 1), (('montt', 'montt'), 2)]


## A.4.9

In [47]:
# Calculate 10 most reacurring word pairs

sorted_counts = wp_counts.sortBy(lambda w: w[1], ascending=False)
sorted_counts.take(10)

[(('är', 'is'), 10040),
 (('vi', 'we'), 5531),
 (('jag', 'i'), 5021),
 (('detta', 'this'), 3253),
 (('avslutad.', 'closed.'), 2964),
 (('och', 'and'), 2917),
 (('en', 'a'), 2888),
 (('det', 'it'), 2866),
 (('det', 'that'), 2806),
 (('inte', 'not'), 2651)]

In [50]:
spark_context.stop()